<a href="https://colab.research.google.com/github/nars95/machine_learning/blob/main/hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Librarys

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from keras.layers import InputLayer, Dense, Flatten
from keras.models import Sequential
from keras.utils import plot_model, to_categorical
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from keras.datasets import mnist

# Load and split the data

In [4]:
train, test = mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [5]:
# split the data
data = train[0]
y = train[1]
X_test = test[0]
y_test = test[1]

# int to float
data = data.astype(float)
X_test = X_test.astype(float)

# normalize the data

data = data / 255.0
X_test = X_test / 255.0

In [6]:
X_train, X_val, y_train, y_val = train_test_split(data, y, train_size=0.8, random_state=42)
print(X_train.shape, X_val.shape)

(48000, 28, 28) (12000, 28, 28)


Buil the model using cross_validation

In [12]:
def create_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape =(28,28)):
  model = Sequential()
  model.add(InputLayer(input_shape=input_shape))
  model.add(Flatten())
  for layer in range(n_hidden):
    model.add(Dense(n_neurons, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [8]:
keras_classifier = keras.wrappers.scikit_learn.KerasClassifier(create_model)

<ipython-input-8-f1fe104ded6b>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_classifier = keras.wrappers.scikit_learn.KerasClassifier(create_model)


In [23]:
param_grid = {
    "n_hidden":[0, 1, 2, 3], "n_neurons":[50, 100, 200]
}
rnd_search_cv = RandomizedSearchCV(keras_classifier, param_grid, cv=3)

In [24]:
rnd_search_cv.fit(X_train, y_train, epochs=10,
                  validation_data =(X_val, y_val),
                  callbacks=[keras.callbacks.ModelCheckpoint("model_cv.h5", best_model_only=True),keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/10
1000/1000 [==============================] - 4s 3ms/step - loss: 0.5816 - accuracy: 0.8519 - val_loss: 0.3638 - val_accuracy: 0.9028
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3357 - accuracy: 0.9071 - val_loss: 0.3179 - val_accuracy: 0.9124
Epoch 3/10
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3033 - accuracy: 0.9153 - val_loss: 0.2991 - val_accuracy: 0.9180
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2866 - accuracy: 0.9201 - val_loss: 0.2915 - val_accuracy: 0.9193
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2768 - accuracy: 0.9236 - val_loss: 0.2874 - val_accuracy: 0.9210
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2694 - accuracy: 0.9257 - val_loss: 0.2863 - val_accuracy: 0.9218
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2631 - accuracy: 0.9272 - val_loss: 0.2831 - val_accuracy:

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fe55fb53760>,
                   param_distributions={'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': [50, 100, 200]})

In [27]:
rnd_search_cv.best_params_

{'n_neurons': 200, 'n_hidden': 2}

In [25]:
model = keras.models.load_model('/content/model_cv.h5')

In [26]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1037 - accuracy: 0.9766


[0.10369467735290527, 0.9765999913215637]